In [3]:
import numpy
import cv2
import warnings
warnings.filterwarnings('ignore')

In [12]:
file = 'vtest.avi'

cap = cv2.VideoCapture(file)

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH) + 0.5)
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT) + 0.5)
size = (width, height)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('my_video.avi', fourcc, 20.0, size)

# Read two frames to detect the movement
ret, frame1 = cap.read()
ret, frame2 = cap.read()

while True:
    
    diff = cv2.absdiff(frame1, frame2) # Find the absolute difference between the frames
    gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5,5), 0)
    # Find the threshold
    _, thresh = cv2.threshold(blur, 20, 255, cv2.THRESH_BINARY)
    dilated = cv2.dilate(thresh, None, iterations = 3)
    contours, _ = cv2.findContours(dilated, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    # Draw a Rectangle in the detected contours
    for contour in contours:
        (x, y, w, h) = cv2.boundingRect(contour)
        
        if cv2.contourArea(contour)<900:
            continue
        cv2.rectangle(frame1, (x, y), (x+w, y+h), (0,255,0), 2)
        cv2.putText(frame1, "Status: {}".format('Movement'), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 
                   1, (0,0,255), 3)
    #cv2.drawContours(frame1, contours, -1, (0,255,0), 2)
    
    cv2.imshow('Feed',frame1)
    out.write(frame1)
    frame1 = frame2
    
    ret, frame2 = cap.read()
    
    # write the flipped frame
    
    
    if cv2.waitKey(40) == 27:
        break

cap.release()
out.release()
cv2.destroyAllWindows()